In [ ]:
import pandas as pd
import plotly.graph_objects as go
lll

In [2]:
df = pd.read_csv("runs.csv", parse_dates=["timestamp"])
df["speedup"] = df["sequential_time"] / df["parallel_time"]
df.head()

,timestamp,ml_model_name,num_workers,num_rows,sequential_time,parallel_time,speedup
0,2025-12-01 11:45:58,LogisticRegression,16,1000000,1.178123,4.569433,0.257827
1,2025-12-01 11:58:45,LogisticRegression,4,1000000,1.416777,5.311674,0.266729
2,2025-12-01 11:58:55,LogisticRegression,8,1000000,1.438387,7.322231,0.196441
3,2025-12-01 11:59:17,LogisticRegression,16,1000000,1.694561,18.988636,0.089241
4,2025-12-01 11:59:54,LogisticRegression,32,1000000,1.888945,33.847085,0.055808


In [6]:
df_median_speedup = df.groupby(["ml_model_name","num_workers","num_rows"]).median()["speedup"].reset_index()
df_median_speedup.pivot_table(
    index=["ml_model_name", "num_workers"],
    columns="num_rows",
    values="speedup"
).reset_index()

num_rows,ml_model_name,num_workers,1000000,10000000,100000000
0,LogisticRegression,1,0.159222,0.622599,NaN
1,LogisticRegression,2,0.180722,1.047393,NaN
2,LogisticRegression,4,0.266729,1.555571,2.887347
3,LogisticRegression,8,0.261435,1.981487,3.825929
4,LogisticRegression,16,0.233003,1.686715,4.765117
5,LogisticRegression,32,0.085022,1.093716,4.838968


In [5]:
# Pivot the data
pivot = df_median_speedup.pivot(
    index="num_workers",
    columns="num_rows",
    values="speedup"
)

fig = go.Figure(
    data=[
        go.Surface(
            x=pivot.columns.values,        # num_rows,
            y=pivot.index.values,          # num_workers,
            z=pivot.values,                # speedup,
            colorscale="Viridis"
        )
    ]
)

fig.update_layout(
    title="Median Speedup Surface",
    autosize=True,
    scene=dict(
        xaxis_title="Number of Rows",
        yaxis_title="Number of Workers",
        zaxis_title="Median Speedup",
    ),
    margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()
